In [11]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [12]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [13]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [14]:
# shuffle up the words
import random
random.seed(42)
random.shuffle(words)

In [15]:
# build the dataset
block_size = 8 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 8]) torch.Size([182625])
torch.Size([22655, 8]) torch.Size([22655])
torch.Size([22866, 8]) torch.Size([22866])


In [16]:
for x,y in zip(Xtr[:20], Ytr[:20]):
  print(''.join(itos[ix.item()] for ix in x), '-->', itos[y.item()])

........ --> y
.......y --> u
......yu --> h
.....yuh --> e
....yuhe --> n
...yuhen --> g
..yuheng --> .
........ --> d
.......d --> i
......di --> o
.....dio --> n
....dion --> d
...diond --> r
..diondr --> e
.diondre --> .
........ --> x
.......x --> a
......xa --> v
.....xav --> i
....xavi --> e


In [17]:
# Near copy paste of the layers we have developed in Part 3

# -----------------------------------------------------------------------------------------------
class Linear:
  
  def __init__(self, fan_in, fan_out, bias=True):
    self.weight = torch.randn((fan_in, fan_out)) / fan_in**0.5 # note: kaiming init
    self.bias = torch.zeros(fan_out) if bias else None
  
  def __call__(self, x):
    self.out = x @ self.weight
    if self.bias is not None:
      self.out += self.bias
    return self.out
  
  def parameters(self):
    return [self.weight] + ([] if self.bias is None else [self.bias])

# -----------------------------------------------------------------------------------------------
class BatchNorm1d:
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.momentum = momentum
    self.training = True
    # parameters (trained with backprop)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
    # buffers (trained with a running 'momentum update')
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    if self.training:
      if x.ndim == 2:
        dim = 0
      elif x.ndim == 3:
        dim = (0,1)
      xmean = x.mean(dim, keepdim=True) # batch mean
      xvar = x.var(dim, keepdim=True) # batch variance
    else:
      xmean = self.running_mean
      xvar = self.running_var
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    # update the buffers
    if self.training:
      with torch.no_grad():
        self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
        self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

# -----------------------------------------------------------------------------------------------
class Tanh:
  def __call__(self, x):
    self.out = torch.tanh(x)
    return self.out
  def parameters(self):
    return []
  
# -----------------------------------------------------------------------------------------------
class Embedding:

  def __init__(self, num_embeddings, embedding_dim):
    self.weight = torch.randn((num_embeddings, embedding_dim))
    # self.out = None

  def __call__(self, ix):
    self.out = self.weight[ix]
    return self.out
  
  def parameters(self):
    return [self.weight]
  
# -----------------------------------------------------------------------------------------------
class FlattenConsecutive:
  """ flattent the first last two dimensions of the tensor based on n"""
  def __init__(self, n):
    self.n = n

  def __call__(self, x): # x --> embedding
    B, T, C = x.shape
    x = x.view(B, T // self.n, C * self.n) # flatten the last two dimensions
    if T//self.n == 1:
      x = x.squeeze(1)
    # self.out = x.view(x.shape[0], -1)
    self.out = x
    return self.out
  
  def parameters(self):
    return []
  
# -----------------------------------------------------------------------------------------------
class Sequential:
  def __init__(self, layers):
    self.layers = layers

  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    self.out = x
    return self.out
  
  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]
  

In [18]:
torch.manual_seed(42); # seed rng for reproducibility

In [31]:
# original network
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP


# C = torch.randn((vocab_size, n_embd))
model = Sequential([
  Embedding(vocab_size, n_embd), # embedding layer
  FlattenConsecutive(2), Linear(n_embd * 2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  FlattenConsecutive(2), Linear(n_hidden * 2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  FlattenConsecutive(2), Linear(n_hidden * 2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(n_hidden, vocab_size),
])

# parameter init
with torch.no_grad():
  model.layers[-1].weight *= 0.1 # last layer make less confident

parameters = model.parameters()
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

170897


In [32]:
ix = torch.randint(0, Xtr.shape[0], (4,))
Xb, Yb = Xtr[ix], Ytr[ix]
logits = model(Xb)
print(Xb.shape)
Xb

torch.Size([4, 8])


tensor([[ 0,  0,  0,  0, 12, 15, 25,  1],
        [ 0,  0,  1, 12,  5, 24, 26,  1],
        [ 0,  0, 10, 15, 11, 21,  2,  1],
        [ 0,  0,  1, 22,  1, 14, 20,  8]])

In [33]:
for layer in model.layers:
    print(layer.__class__.__name__, " : ", tuple(layer.out.shape))

    # print(dir(layer))
    # break

Embedding  :  (4, 8, 10)
FlattenConsecutive  :  (4, 4, 20)
Linear  :  (4, 4, 200)
BatchNorm1d  :  (4, 4, 200)
Tanh  :  (4, 4, 200)
FlattenConsecutive  :  (4, 2, 400)
Linear  :  (4, 2, 200)
BatchNorm1d  :  (4, 2, 200)
Tanh  :  (4, 2, 200)
FlattenConsecutive  :  (4, 400)
Linear  :  (4, 200)
BatchNorm1d  :  (4, 200)
Tanh  :  (4, 200)
Linear  :  (4, 27)


In [ ]:
model.layers[0].out.shape # embedding layer

torch.Size([4, 8, 10])

In [ ]:
model.layers[1].out.shape # flatten layer

torch.Size([4, 80])

In [ ]:
model.layers[2].out.shape # linear layer

torch.Size([4, 200])

In [ ]:
(torch.randn(4,80) @ torch.randn(80, 200) + torch.randn(200)).shape # matrix multiplication ryt now

torch.Size([4, 200])

In [ ]:
(torch.randn(4, 5, 6, 80) @ torch.randn(80, 200) + torch.randn(200)).shape

torch.Size([4, 5, 6, 200])

In [ ]:
# (1 2) (3 4) (5 6) (7 8)  # process all the bigrams in parallel.
# we only want two characters to come in.
(torch.randn(4, 4, 20)  @ torch.randn(20, 200) + torch.randn(200)).shape

torch.Size([4, 4, 200])

In [ ]:
list(range(10))[::2]

[0, 2, 4, 6, 8]

In [ ]:
e = torch.randn(4, 8, 10) # goal: want this to be (4, 4, 20) where consecutive characters(10d vectiors) are together


In [ ]:
e[:, ::2, :][0]

tensor([[-2.3087, -0.9386, -1.4798,  1.6941,  0.3415,  0.9090,  0.1128,  0.3336,
          0.8422, -0.9001],
        [ 2.1564,  2.2873,  0.5820,  0.9450,  0.2252, -0.0453, -0.1370, -2.4657,
          0.9312, -0.1134],
        [-0.1129,  1.8691,  0.8729, -0.4268,  0.8660,  0.7447,  1.0083, -0.1496,
          0.7476,  0.2390],
        [-0.8722,  0.7643, -1.7641, -0.2202,  0.4567, -1.0165,  0.3727,  0.9812,
          0.9573,  0.1915]])

In [ ]:
e[:, 1::2, :][0]

tensor([[ 1.4415,  0.4449,  2.1014, -0.5588,  0.3988,  0.6231, -1.8995, -0.7779,
          1.1277,  0.5441],
        [-0.1288,  0.8333,  0.4062, -1.0567,  0.2665, -0.1885,  1.3110,  0.9927,
         -1.4903, -0.5466],
        [-0.1744,  0.6139,  0.6712, -1.0645, -0.3517,  1.6924, -0.4610, -1.0880,
         -0.1881,  1.8550],
        [-0.8099, -0.8015,  0.5232,  0.3842, -0.4291, -0.5256,  0.1525, -0.5406,
         -0.4012, -1.3024]])

In [ ]:
explicit = torch.cat([e[:, ::2, :], e[:, 1::2, :]], dim=2) # this is the final shape we want

In [ ]:
e.shape

torch.Size([4, 8, 10])

In [ ]:
em = e.view(4, 4, 20)

In [ ]:
(em == explicit).all()

tensor(True)

In [ ]:
# same optimization as last time
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):

    # Mini batches
    ix = torch.randint(0, Xtr.shape[0], (batch_size,)) # random batch of indices
    Xb, Yb = Xtr[ix], Ytr[ix]

    # forward pass
    logits = model(Xb) # logits
    # for layer in layers:
    #     x = layer(x)
    loss = F.cross_entropy(logits, Yb) # cross-entropy loss

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # update
    lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
    for p in parameters:
        p.data += -lr * p.grad

    # track stats
    if i % 10000 == 0: # print every once in a while
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())
    break 

      0/ 200000: 3.3093


In [ ]:
#using view we can make it 2d 
plt.plot(torch.tensor(lossi).view(-1, 1000).mean(1))

RuntimeError: shape '[-1, 1000]' is invalid for input of size 1

In [ ]:
for layer in model.layers:
    layer.training = False # turn off training mode because batchnorm is in training mode

In [ ]:
# evaluate the loss
@torch.no_grad() # this decorator disables gradient tracking inside pytorch
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  logits = model(x)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 3.274365186691284
val 3.2746033668518066


In [ ]:
for _ in range(20):

    op = []
    context = [0] * block_size
    while True:
        
        logits =  model(torch.tensor([context]))
        probs = F.softmax(logits, dim=1)

        ix = torch.multinomial(probs, num_samples=1).item()

        context = context[1:] + [ix] 
        op.append(ix)
        if ix == 0:
            break
    
    print("".join(itos[i] for i in op))

iwznxjqtkwzfdkomolsbpxycwsvtldirhmdhrbteybzcpstyjgijlnkxywyrutplzcqxgpx.
fzfnqftfgrfoozlsyru.
bclfe.
uqooskpbkkdoacsjqfcoenagpavsekuatmglwajtmblskcjqumxxqfkx.
pfbhrcniurgnhlirl.
dquv.
uhop.
tywpahxitzrnqjljgapvbcxwkiwrsxchclkrrrvsavxxgnrixmyapaaswksklxlmkl.
gzanmghvlzumtqtijzvppldteakew.
ir.
ggygbcy.
bsahebneg.
hwixsoaewoowrywyplgkcjupknzwubkektqzonaporirzdigzstivqlkhvlettjcgotbsfychcrkvfmenyjlcnqcqeajtvvgtr.
yte.
gbpwctxsmbrwxlthblazmcftsgnpvqrfmobipzthvbwitgavnz.
.
kgfzmapepprilqqvo.
kljdvqwzszw.
ouknwljnrkxhfeqpcbkxczndqqsfodoyqecysln.
wjriwg.
